In [4]:
%matplotlib inline

In [5]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [7]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [8]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


In [12]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


In [11]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [12]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [13]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [14]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.3619, -0.4383, -0.1442,  0.0192,  0.2954,  0.2532,  0.0101,  0.1517,
         -0.7363, -0.0783, -0.3528,  0.2971, -0.2994,  0.1889, -0.0465, -0.0643,
          0.5521,  0.7546,  0.3357,  0.0916],
        [ 0.5081, -0.4882, -0.0421, -0.1530,  0.2651,  0.3612,  0.3170,  0.0252,
         -0.5674, -0.3302,  0.0054,  0.3544, -0.2292,  0.2231, -0.1889, -0.1133,
          0.4390,  0.4113,  0.2872,  0.0054],
        [ 0.6177, -0.4095, -0.1539,  0.2486,  0.7899,  0.3194,  0.0872,  0.1299,
         -0.7683, -0.2850,  0.0171,  0.3070, -0.2494,  0.4497,  0.0295,  0.3254,
          0.6013,  0.6649,  0.1127,  0.0663]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.3619, 0.0000, 0.0000, 0.0192, 0.2954, 0.2532, 0.0101, 0.1517, 0.0000,
         0.0000, 0.0000, 0.2971, 0.0000, 0.1889, 0.0000, 0.0000, 0.5521, 0.7546,
         0.3357, 0.0916],
        [0.5081, 0.0000, 0.0000, 0.0000, 0.2651, 0.3612, 0.3170, 0.0252, 0.0000,
         0.0000, 0.0054, 0.3544, 0.0000, 0.2231, 0.00

In [15]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3, 28, 28)
logits = seq_modules(input_image)

In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
print("Model structure: ", model, '\n\n')

for name, param in model.named_parameters():
    print(f"layer: {name} | Size: {param.size()} | Values: {param[:2]}")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values: tensor([[-0.0018, -0.0310,  0.0083,  ...,  0.0073,  0.0273,  0.0244],
        [-0.0064, -0.0353,  0.0309,  ..., -0.0140,  0.0340,  0.0092]],
       grad_fn=<SliceBackward0>)
layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values: tensor([-0.0293,  0.0215], grad_fn=<SliceBackward0>)
layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values: tensor([[ 0.0327, -0.0333,  0.0219,  ..., -0.0035,  0.0060, -0.0138],
        [-0.0189, -0.0263,  0.0155,  ...,  0.0100,  0.0010,  0.0301]],
       grad_fn=<SliceBackward0>)
layer: linear_relu_stack.